In [3]:
import joblib
import numpy as np
import time

# ---------------------------
# CONFIG
# ---------------------------
FRAME_LEN = 6  # must match training
FEATURES = ["gyro_x", "gyro_y", "gyro_z", "acc_x", "acc_y", "acc_z"]

# ---------------------------
# LOAD TRAINED COMPONENTS
# ---------------------------
clf = joblib.load("beta_gesture_model.joblib")
scaler = joblib.load("beta_gesture_scaler.joblib")
encoder = joblib.load("beta_gesture_encoder.joblib")

# ---------------------------
# HELPER FUNCTION
# ---------------------------
'''
def preprocess_gesture(frames, frame_len=FRAME_LEN):
    """
    Preprocess a gesture (list of [gyro_x, gyro_y, gyro_z, acc_x, acc_y, acc_z])
    by padding/truncating and flattening to match training input format.
    """
    data = np.array(frames)

    # pad/truncate
    if data.shape[0] < frame_len:
        pad_len = frame_len - data.shape[0]
        data = np.vstack([data, np.zeros((pad_len, data.shape[1]))])
    elif data.shape[0] > frame_len:
        data = data[:frame_len, :]

    # flatten and scale
    flat = data.flatten().reshape(1, -1)
    flat = scaler.transform(flat)
    return flat
'''
# ---------------------------
# TEST GESTURE INPUT
# ---------------------------
test_gesture_x = [
    [-24.274809, -1.587786, 20.862595, -0.011177, 0.00414, 0.068278],
    [75.35878, -13.206107, 45.916031, -0.054557, 0.026034, 0.125],
    [58.572517, 66.938934, -11.854961, -0.015317, 0.036047, 0.151142],
    [18.061069, -21.229008, -98.435112, 0.056018, 0.032962, 0.130006],
    [-66.251907, -35.61832, -56.694656, 0.025195, 0.006441, 0.057317],
    [-85.145035, -11.488549, 44.38168, -0.040647, -0.036155, 0.066113]
]

test_gesture_c = [
    [-11.541985, 10.122137, -40.305344, 0.058616, 0.001245, 0.066546],
    [99.618324, 30.038168, -26.648855, 0.004168, 0.034098, 0.154984],
    [50.091602, -20.083969, 85.39695, -0.080401, 0.03648, 0.145513],
    [-68.832062, -10.526717, 73.328247, -0.024302, 0.018862, 0.073311],
    [-99.610687, -9.297709, -27.167938, 0.019864, -0.0082, 0.066302]

] 

# ---------------------------
# RUN PREDICTION
# ---------------------------

'''
part1 = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0]
part2 = [2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
part3 = [3.0, 4.0, 5.0, 6.0, 7.0, 8.0]
part4 = [4.0, 5.0, 6.0, 7.0, 8.0, 9.0]
part5 = [5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
part6 = [6.0, 7.0, 8.0, 9.0, 10.0, 11.0]

while len(buffer) < 6:
    if len(buffer) == 0:
        packet = part1
    elif len(buffer) == 1:
        packet = part2
    elif len(buffer) == 2:
        packet = part3
    elif len(buffer) == 3:
        packet = part4
    elif len(buffer) == 4:
        packet = part5
    elif len(buffer) == 5:
        packet = part6
    
    # Append the incoming packet to the buffer
    buffer.append(packet)
'''

#----------good stuff---------


RECORD_DURATION = 2.0  # seconds
start_time = time.time()
buffer = []

while time.time() - start_time < RECORD_DURATION:
    if len(buffer) >= FRAME_LEN:
        # Stop collecting once we have enough frames
        break
    imu_line = ser.readline().decode('utf-8', errors='ignore').strip()
    parts = imu_line.split(",")
    ax, ay, az, gx, gy, gz = parts
    packet = [float(ax), float(ay), float(az), float(gx), float(gy), float(gz)]
    buffer.append(packet)
    print(len(buffer))


user_data = np.array(buffer)

print(user_data)

X_new = preprocess_gesture(user_data)

# Run prediction
probs = clf.predict_proba(X_new)[0]
max_prob = np.max(probs)
pred_class = np.argmax(probs)

THRESHOLD = 0.6
if max_prob < THRESHOLD:
    gesture = "FAILED"
else:
    gesture = encoder.inverse_transform([pred_class])[0]

print(f"Predicted gesture: {gesture} | Confidence: {max_prob:.3f}")
print("Class probabilities:")
for cls, prob in zip(encoder.classes_, probs):
    print(f"  {cls}: {prob:.3f}")

#----------good stuff-------

NameError: name 'imu_line' is not defined